<a href="https://colab.research.google.com/github/LEEHYUNJI94/Python_Basics/blob/master/%EC%9E%AC%EB%AC%B4%EC%9E%AC%ED%91%9C%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 재무재표 파일 자동 다운로드 
### url get 1

In [ ]:
import requests,urllib
from bs4 import BeautifulSoup

dart_url='http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20200814000957&dcm_no=7443370&lang=ko'

resp=requests.get(dart_url)
resp #<Response [200]>
resp.content #b'' ->자동수집을 막아놓았기때문에 수집 X

### url get 1-1
#### 크롤링 접속 차단 해제- User-Agent사용


In [ ]:
user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
resp=requests.get(dart_url, headers={'user-agent': user_agent})
resp.content

b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00>\x00\x03\x00\xfe\xff\t\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00_\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00^\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\

#### BytesIO사용 후 excel 정보읽어 오기

In [ ]:
from io import BytesIO
import pandas as pd
#BytesIO를 통해 알아 볼 수 있는 언어로 변환
table=BytesIO(resp.content)
#엑셀정보 확인-시트 연결 재무상태표, 5개줄 skip
save_bs_ho=pd.read_excel(table,sheet_name="연결 재무상태표", skiprows=5)
#엑셀정보 저장 (저장경로:.py가 있는 폴더)
save_bs_ho.to_csv("연결재무상태표_효성.csv", encoding="utf-8")

###for문으로 sheet분리 
#### -Excel tab에 있는 sheet 분리 후 csv 파일저장

In [ ]:
sheets=["연결 재무상태표","연결 포괄손익계산서","현금흐름표"]
for sheet in sheets:
  save_fin_info=pd.read_excel(table, sheet_name=sheet, skiprows=5)
  save_fin_info.to_csv(sheet+".csv", encoding="utf-8")

### URL 주소를 수기로 변경해 재무재표 다운받기 1
-2중 for문 사용



In [ ]:
import requests
from io import BytesIO
import pandas as pd
url_1='http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20200515001628&dcm_no=7341859&lang=ko'
url_2='http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20200814000957&dcm_no=7443370&lang=ko'
urls=[url_1, url_2]
sheets=["연결 재무상태표","연결 포괄손익계산서","현금흐름표"]
user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
#기간을 넣어 파일이름다르게 설정 
#url을 통해 파일 다운로드
for period , url in zip(['202005','202008'],urls):  
  resp=requests.get(url, headers={'user-agent':user_agent})
  table=BytesIO(resp.content)
#엑셀을 읽어 csv파일로 저장  
  for sheet in sheets:
    save=pd.read_excel(table, sheet_name=sheet, skiprows=5)
    save.to_csv(period + sheet+".csv", encoding='utf-8')


-함수+for문사용


In [ ]:
import requests
from io import BytesIO
import pandas as pd
user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'

#함수 정의: 파일 다운로드 후 원하는 방식대로 편집 후 csv로 저장
def download_save (url,period,company):
    resp=requests.get(url,headers={'user-agent':user_agent})
    table=BytesIO(resp.content)
    for sheet in ["연결 재무상태표","연결 포괄손익계산서","현금흐름표"]:
      save=pd.read_excel(table, sheet_name=sheet, skiprows=5)
      save.to_csv(period+company+sheet+".csv", encoding='utf-8')

#함수실행
url_1='http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20200515001628&dcm_no=7341859&lang=ko'
url_2='http://dart.fss.or.kr/pdf/download/excel.do?rcp_no=20200814000957&dcm_no=7443370&lang=ko'
urls=[url_1, url_2]
for period , url in zip(['202005','202008'],urls):
  download_save(url,period,'효성중공업')

### URL 주소를 파일로 불러와 재무재표 다운받기 2
#### 1. url file import

In [ ]:
from google.colab import files
files.upload()

Saving url_inform.csv to url_inform.csv


{'url_inform.csv': b'period,rcp_no,dcm_no\r\n2020.05,20200515001628 ,7341859\r\n2020.08,20200814000957 ,7443370\r\n'}

#### 2. 파일로 데이터 불러오기

In [ ]:
import pandas as pd
import requests 
from io import BytesIO

#함수정의
def download_save_2 (rcp_no,dcm_no,period,company):
  url='http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko'.format(rcp_no,dcm_no)
  user_agent="'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'"
  resp=requests.get(url, headers={'user-agent':user_agent})
  table=BytesIO(resp.content)
  for sheet in ["연결 재무상태표","연결 포괄손익계산서","현금흐름표"]:
    down=pd.read_excel(table,sheet_name=sheet,skiprwos=5)
    down.to_csv(str(period)+"_"+sheet+"_"+company+".csv" ,encoding='utf-8')
#함수실행
url_inform=pd.read_csv("url_inform.csv")
url_inform['period'].values #array([2020.05, 2020.08])
url_inform['period'].index  #RangeIndex(start=0,stop=2,step=1) 0부터 1간격으로 길이2
#파일에 있는 컬럼 url_inform 파일의 period컬럼
for period, rcp_no, dcm_no in zip(url_inform['period'].values,url_inform['rcp_no'].values,url_inform['dcm_no'].values):
  download_save_2 (rcp_no,dcm_no,period,'효성중공업')

### URL 주소를 딕셔너리형태로 불러와 재무재표 다운받기 3

In [ ]:
url_dict={
    '2020.05': [20200515001628,7341859],
    '2020.08': [20200814000957,7443370]
}

for period ,value in url_dict.items():
  rcp_no = value[0]
  dcm_no = value[1]
  download_save_2 (rcp_no, dcm_no,period,'효성중공업')